In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import requests
import json

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

In [3]:
# url to API
url = 'https://ies-midterm.soulution.rocks/'

In [45]:
# post request to login to API
req = requests.post(url + "login", json={"cuni":"96640370"})
print(req.text)
req_json = req.json()

{"data": {"personal_code": "f498a093", "dataset_ids": ["775352ba11", "e2beb850b2", "81395feb08", "40138ff3b8", "ef0bccd7cc"]}, "message": "Hello, Karl\u00ed\u010dek Ond\u0159ej! Good luck with the midterm. GET your data set from /data/file_id endpoint; use the codes you received in the data response as the file_id parameter! Print the personal code in your notebook."}


In [47]:
# printing personal code
print("My personal code is: " + str(req_json['data']['personal_code']))

My personal code is: f498a093


In [49]:
# function for loading data
def load_data(dataset_id):
    r = requests.get(f"{url}data/{dataset_id}")
    return r.json()

In [95]:
# loading data
warning_message = 'Error [503] - Sorry, I just slipped and fell. Thank God, I am made of silicon. You need to work this out.'
datasets = []
for idd in req_json['data']['dataset_ids']:
    successful = False
    # loading the same dataset until it is successful
    while not successful:
        data = load_data(idd)
        if data['message'] == warning_message:
            # if the data weren't loaded continue the while
            continue
        else:
            # apend data to datasets list and break the while loop with successful
            datasets.append(data)
            successful = True

In [129]:
# creating pandas dataframes
df_all = pd.DataFrame(datasets[0]['data']['data'])
df_all = df_all.melt(id_vars=['Date'], var_name="type", value_name=datasets[0]['data']['company']).set_index(['Date', 'type'])

for data in datasets[1:]:
    df = pd.DataFrame(data['data']['data'])
    df = df.melt(id_vars=['Date'], var_name="type", value_name=data['data']['company']).set_index(['Date', 'type'])
    
    df_all = df_all.join(df, how="outer")

df_all.tail(n=20)

SMIN                 IAG  \
Date       type                                                
2020-11-17 Open        37.40999984741211  3.4700000286102295   
           Volume                  16600             6726700   
2020-11-18 Adj.Close  37.599998474121094   3.359999895095825   
           Close      37.599998474121094   3.359999895095825   
           High       37.900001525878906   3.450000047683716   
           Low         37.59000015258789  3.3399999141693115   
           Open       37.900001525878906   3.450000047683716   
           Volume                  31500             5219400   
2020-11-19 Adj.Close  37.970001220703125   3.369999885559082   
           Close      37.970001220703125   3.369999885559082   
           High                     38.0  3.4100000858306885   
           Low        37.790000915527344   3.309999942779541   
           Open       37.790000915527344   3.319999933242798   
           Volume                  18100             5621800   
2020-11-20 Adj.Close   38.16999816894531  3.4100000858306885   
           Close       38.16999816894531  3.4100000858306885   
           High         38.2400016784668   3.509999990463257   
           Low         38.16999816894531  3.4000000953674316   
           Open       38.189998626708984  3.4200000762939453   
           Volume                  10500             6329600   

                                     CCH                EVR CRDA  
Date       type                                                   
2020-11-17 Open       18.920000076293945  87.87999725341797  NaN  
           Volume                 491872             309400  NaN  
2020-11-18 Adj.Close   18.81999969482422  88.30999755859375  NaN  
           Close       18.81999969482422  88.30999755859375  NaN  
           High       19.239999771118164  90.69000244140625  NaN  
           Low        18.739999771118164  88.27999877929688  NaN  
           Open       18.950000762939453  89.18000030517578  NaN  
           Volume                 756785             334200  NaN  
2020-11-19 Adj.Close  19.100000381469727  88.48999786376953  NaN  
           Close      19.100000381469727  88.48999786376953  NaN  
           High                     19.5  88.55000305175781  NaN  
           Low        18.649999618530273              87.25  NaN  
           Open       18.850000381469727  87.81999969482422  NaN  
           Volume                 725184             166300  NaN  
2020-11-20 Adj.Close  19.389999389648438  88.30999755859375  NaN  
           Close      19.389999389648438  88.30999755859375  NaN  
           High                     19.5  88.56999969482422  NaN  
           Low         19.06999969482422  87.16000366210938  NaN  
           Open       19.290000915527344   87.9800033569336  NaN  
           Volume                1020227             196500  NaN

,,SMIN
Date,type,
2019-01-02,Open,38.38999938964844
2019-01-03,Open,37.97999954223633
2019-01-04,Open,38.150001525878906
2019-01-07,Open,38.61000061035156
2019-01-08,Open,38.66999816894531
...,...,...
2020-11-17,Volume,16600
2020-11-18,Volume,31500
2020-11-19,Volume,18100


4